Part 1:

Combinations:

* 2 segments: **1** (C, F)
* 3 segments: **7** (A, C, F)
* 4 segments: **4** (A, C, D, F)
* 5 segments: **2** (A, C, D, E, G), **3** (A, C, D, F, G), **5** (A, B, D, F, G)
* 6 segments: **0** (A, B, C, E, F, G), **6** (A, B, D, E, F, G), **9** (A, B, C, D, F, G)
* 7 segments: **8** (A, B, C, D, E, F, G)

In [1]:
def parse_input_part_1(input_file: str) -> list[str]:
    with open(input_file) as signal_patterns_and_output:
        return [output for line in signal_patterns_and_output.readlines()
                for output in line.strip().split(' | ')[1].split(' ')]

In [2]:
def count_unique_combinations(screen_output_file: str) -> int:
    rendered_output = parse_input_part_1(screen_output_file)
    unique_numbers_found = 0
    for rendered_number in rendered_output:
        if any(len(rendered_number) == n for n in [2, 3, 4, 7]):
            unique_numbers_found += 1
    return unique_numbers_found

In [3]:
count_unique_combinations('practise_input.txt')

26

In [4]:
count_unique_combinations('real_input.txt')

342

Part 2:

After some investigation, we have found that after updating mappings for the unique combinations, the remaining mappings will always have two letters that map to c or f, two that map to e or g, two that map to b or d and one that maps to a.

We can use 0 to resolve which maps to b or d. We can use 6 to resolve which maps to f or c. We can then use any number to resolve e vs g. 

In [5]:
wire_to_number: dict[tuple[str]:str] = {('a', 'b', 'c', 'e', 'f', 'g'): '0', ('c', 'f'): '1', ('a', 'c', 'd', 'e', 'g'): '2', 
                    ('a', 'c', 'd', 'f', 'g'): '3', ('b', 'c', 'd', 'f'): '4',
                    ('a', 'b', 'd', 'f', 'g'): '5', ('a', 'b', 'd', 'e', 'f', 'g'): '6',
                    ('a', 'c', 'f'): '7', ('a', 'b', 'c', 'd', 'e', 'f', 'g'): '8',
                    ('a', 'b', 'c', 'd', 'f', 'g'): '9'}

In [6]:
unique_length_to_wire: dict[int:set[str]] = {2:{'c', 'f'}, 3:{'a', 'c', 'f'}, 4:{'b', 'c', 'd', 'f'}}

In [7]:
ALL_SEGMENTS: set[str]= {'a', 'b', 'c', 'd', 'e', 'f', 'g'}

In [8]:
def parse_input_part_2(input_file: str) -> list[str]:
    with open(input_file) as signal_patterns_and_output:
        return [([{*pattern} for pattern in [*patterns.split()]], [{*output} for output in [*outputs.split()]]) 
                    for line in signal_patterns_and_output.readlines()
                    for patterns, outputs in [line.strip().split(' | ')]]

In [9]:
def initialise_potential_mappings() -> dict:
    letters = list(map(chr, range(ord('a'), ord('g')+1)))
    mappings = {letter:set(letters) for letter in letters}
    return mappings

In [10]:
def update_mappings(current_mappings: dict[str:set], mappings_to_update: set[str], new_information: set[str]):
    for key, mapping in current_mappings.items():
        if key in mappings_to_update:
            current_mappings[key] = mapping - (ALL_SEGMENTS - new_information)
        else:
            current_mappings[key] = mapping - new_information

In [11]:
def update_mappings_for_unique_segment_lengths(signal_patterns: list[str], mappings: dict[str:set]) -> dict[str:set]:
    for pattern in signal_patterns:
        if any(len(pattern) == n for n in [2, 3, 4]):
            update_mappings(mappings, pattern, unique_length_to_wire[len(pattern)])

In [12]:
def update_mapping_for_final_matches(mappings: dict[str:set], resolved_key: set[str], 
                                        resolved_mapping: str, unresolved_key: set[str],
                                        unresolved_mapping: str):
    mappings[('').join(resolved_key)] = resolved_mapping
    mappings[('').join(unresolved_key - resolved_key)] = unresolved_mapping

In [13]:
def resolve_remaining_matches(signal_patterns: list[str], mappings: dict[str:set]):
    backwards_map = {'a': {key1} for key1, value1 in mappings.items() if value1 == {'a'}}
    backwards_map.update({('').join(sorted(value1)): {key1, key2} 
                            for key1, value1 in mappings.items() 
                            for key2, value2 in mappings.items() 
                            if value1 == value2 and key1 != key2})
    without_bd = {letter for key, mapping in backwards_map.items() for letter in mapping if 'b' not in key}
    without_cf = {letter for key, mapping in backwards_map.items() for letter in mapping if 'c' not in key}
    without_eg = {letter for key, mapping in backwards_map.items() for letter in mapping if 'e' not in key}
    mappings[('').join(backwards_map['a'])] = 'a'
    for pattern in signal_patterns:
        if len(pattern) == 6:
            if not backwards_map['bd'] < pattern:
                maps_to_b = pattern - without_bd
                update_mapping_for_final_matches(mappings, maps_to_b, 'b', backwards_map['bd'], 'd')
            if not backwards_map['cf'] < pattern:
                maps_to_f = pattern - without_cf
                update_mapping_for_final_matches(mappings, maps_to_f, 'f', backwards_map['cf'], 'c')
            if not backwards_map['eg'] < pattern:
                maps_to_g = pattern - without_eg
                update_mapping_for_final_matches(mappings, maps_to_g, 'g', backwards_map['eg'], 'e')

In [14]:
def find_final_mappings(signal_patterns: list[str]) -> dict[str:set]:
    mappings = initialise_potential_mappings()
    update_mappings_for_unique_segment_lengths(signal_patterns, mappings)
    resolve_remaining_matches(signal_patterns, mappings)
    return mappings

In [15]:
def convert_output(outputs: list[str], final_mappings: dict[str:str]) -> list[str]:
    converted_output = []
    for output in outputs:
        converted_number = set()
        for letter in output:
            converted_number.add(final_mappings[letter])
        converted_output.append(converted_number)
    return converted_output

In [16]:
def build_number(output: list[str]) -> int:
    number = ''
    for item in output:
        number += wire_to_number[tuple(sorted(item))]
    return int(number)


In [17]:
def calculate_answer(input_file: str) -> int:
    total = 0
    signal_patterns_and_output = parse_input_part_2(input_file)
    for signal_pattern, output in signal_patterns_and_output:
        final_mappings = find_final_mappings(signal_pattern)
        converted_output = convert_output(output, final_mappings)
        total += build_number(converted_output)
    return total

In [18]:
calculate_answer('practise_practise_input.txt')

5353

In [19]:
calculate_answer('practise_input.txt')

61229

In [20]:
calculate_answer('real_input.txt')

1068933